In [7]:
import pandas as pd
import numpy  as np

def gen_cycle(stock):
    for qq in ['Close']:
        df = pd.read_excel('E:/Host/Data/Database/from_Wealth/' + stock + '.xlsx', decimal='.', sep = ',', encoding='latin-1', dayfirst=True)
        #change type Time
        df['Time'] =  pd.to_datetime(df['Time'], format='%H:%M:%S')
        df = df.drop_duplicates(['Date', 'Time'], ignore_index = True)

        #mark 12:45, 10:30
        df['12:45'] = np.where(df.Time == '1900-01-01 12:45:00', 1, 0)
        df['10:30'] = np.where(df.Time == '1900-01-01 10:30:00', 1, 0)
        #df[df['12:45']==1]
        df1 = df[(df['12:45']==1)]
        df2 = df[(df['10:30']==1)]

        #merg df1 df2 ---> df3
        df3 = pd.merge(df1, df2, how = 'inner', on = 'Date', suffixes=('', '_x'), right_index = True)
        df3 = df3.drop_duplicates('Date')
        display(df3.shape)
        #df3[df3['Date'] >= '2008-10-23']

        #Создали return df3
        #live 3 columns
        df3['return'] = (df3['Close_x'].shift(-1)/df3['Close']-1)*100
        col = ['Date',
               'Time', 
                'return'
         ]
        df3 = df3[col]
        df3 = df3.drop_duplicates('Date')
        display(df3.shape)

        # Создаем блоки утра namber = 16 
        all_15 = pd.DataFrame()
        for i in range(15, len(df)):
            if df.loc[i]['12:45'] == 1:
                df_15 = df.loc[i-15:i]
                all_15 = all_15.append(df_15)
        display(all_15.shape)        
        display(len(all_15)/16)   

        #create pl-mi
        df3['return_pm'] = np.where(df3['return']>0, 1, 0)

        #create data+16close_morning for DATA
        close = np.array(all_15[qq])
        close = np.reshape(close, (int(len(all_15)/16), 16))
        data = np.array(all_15['Date'])
        data = np.reshape(data, (int(len(all_15)/16), 16))
        data = data[:,-1]
        data_close = pd.concat([pd.DataFrame(data, columns=['Date']), pd.DataFrame(close)], axis=1)
        data_close.shape

        #merg data_close and df3 for DATA ИТОГОВАЯ
        display(df3.shape)
        display(data_close.shape)
        df_merg = pd.merge(df3, data_close, how='inner', on='Date')
        df_merg['Symbol'] = stock
        display(df_merg.shape)
        df_merg.fillna(0, inplace = True)
        display(df_merg.shape)
        df_merg.drop_duplicates(subset = ['Date'], inplace = True)
        display(df_merg.shape)
        display('df_merg')
        display(df_merg)

        return df_merg